In [1]:
import gzip
import collections
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import string
import math
import re
import pandas as pd
from nltk.stem.porter import *
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import csv
from sklearn.model_selection import train_test_split
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')
from sklearn.tree import DecisionTreeRegressor
from collections import defaultdict
from sklearn.svm import SVR

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/bpan/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
# Helper Methods
def readGz(path):
    for l in gzip.open(path, 'rt', encoding='utf-8'):
        if 'null' not in l:
            yield eval(l)

In [3]:
mod = list(readGz("modcloth_final_data.json.gz"))
mod = pd.DataFrame(mod)
mod.head()

,item_id,waist,size,quality,cup size,hips,bra size,category,bust,height,user_name,length,fit,user_id,shoe size,shoe width,review_summary,review_text
0,123373,29,7,5.0,d,38,34,new,36,5ft 6in,Emily,just right,small,991571,NaN,NaN,NaN,NaN
1,123373,31,13,3.0,b,30,36,new,NaN,5ft 2in,sydneybraden2001,just right,small,587883,NaN,NaN,NaN,NaN
2,123373,30,7,2.0,b,NaN,32,new,NaN,5ft 7in,Ugggh,slightly long,small,395665,9.00,NaN,NaN,NaN
3,123373,NaN,21,5.0,dd/e,NaN,NaN,new,NaN,NaN,alexmeyer626,just right,fit,875643,NaN,NaN,NaN,NaN
4,123373,NaN,18,5.0,b,NaN,36,new,NaN,5ft 2in,dberrones1,slightly long,small,944840,NaN,NaN,NaN,NaN


In [4]:
rent = list(readGz("renttherunway_final_data.json.gz"))
rent = pd.DataFrame(rent)
rent.head()

,fit,user_id,bust size,item_id,weight,rating,rented for,review_text,body type,review_summary,category,height,size,age,review_date
0,fit,420272,34d,2260466,137lbs,10,vacation,An adorable romper! Belt and zipper were a lit...,hourglass,So many compliments!,romper,"5' 8""",14,28,"April 20, 2016"
1,fit,273551,34b,153475,132lbs,10,other,I rented this dress for a photo shoot. The the...,straight & narrow,I felt so glamourous!!!,gown,"5' 6""",12,36,"June 18, 2013"
2,fit,360448,NaN,1063761,NaN,10,party,This hugged in all the right places! It was a ...,NaN,It was a great time to celebrate the (almost) ...,sheath,"5' 4""",4,116,"December 14, 2015"
3,fit,909926,34c,126335,135lbs,8,formal affair,I rented this for my company's black tie award...,pear,Dress arrived on time and in perfect condition.,dress,"5' 5""",8,34,"February 12, 2014"
4,fit,151944,34b,616682,145lbs,10,wedding,I have always been petite in my upper body and...,athletic,Was in love with this dress !!!,gown,"5' 9""",12,27,"September 26, 2016"


In [5]:
rent["TextBlob_polarity"] = rent["review_text"].apply(lambda review: TextBlob(review).polarity)
rent["TextBlob_subjectivity"] = rent["review_text"].apply(lambda review: TextBlob(review).subjectivity)

In [6]:
rent["nltk_sentiment_compound_score"] = rent["review_text"].apply(lambda review: SentimentIntensityAnalyzer().polarity_scores(review)["compound"])
rent["nltk_sentiment_category"] = rent["nltk_sentiment_compound_score"].apply(lambda analysis: "positive" if analysis > 0.05 
                                                                                else ("negative" if analysis < -0.05 else "neutral"))

In [7]:
rent["nltk_sentiment_compound_score_summary"] = rent["review_summary"].apply(lambda review: SentimentIntensityAnalyzer().polarity_scores(review)["compound"])
rent["nltk_sentiment_category_summary"] = rent["nltk_sentiment_compound_score_summary"].apply(lambda analysis: "positive" if analysis > 0.05 
                                                                                else ("negative" if analysis < -0.05 else "neutral"))

In [8]:
rent

,fit,user_id,bust size,item_id,weight,rating,rented for,review_text,body type,review_summary,...,height,size,age,review_date,TextBlob_polarity,TextBlob_subjectivity,nltk_sentiment_compound_score,nltk_sentiment_category,nltk_sentiment_compound_score_summary,nltk_sentiment_category_summary
0,fit,420272,34d,2260466,137lbs,10,vacation,An adorable romper! Belt and zipper were a lit...,hourglass,So many compliments!,...,"5' 8""",14,28,"April 20, 2016",0.181548,0.623810,0.9453,positive,0.5057,positive
1,fit,273551,34b,153475,132lbs,10,other,I rented this dress for a photo shoot. The the...,straight & narrow,I felt so glamourous!!!,...,"5' 6""",12,36,"June 18, 2013",0.360667,0.686667,0.8991,positive,0.6333,positive
2,fit,360448,NaN,1063761,NaN,10,party,This hugged in all the right places! It was a ...,NaN,It was a great time to celebrate the (almost) ...,...,"5' 4""",4,116,"December 14, 2015",0.489762,0.570476,0.9333,positive,0.8316,positive
3,fit,909926,34c,126335,135lbs,8,formal affair,I rented this for my company's black tie award...,pear,Dress arrived on time and in perfect condition.,...,"5' 5""",8,34,"February 12, 2014",0.213056,0.461556,0.9559,positive,0.5719,positive
4,fit,151944,34b,616682,145lbs,10,wedding,I have always been petite in my upper body and...,athletic,Was in love with this dress !!!,...,"5' 9""",12,27,"September 26, 2016",-0.054028,0.320417,0.6948,positive,0.7249,positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192457,fit,66386,34dd,2252812,140lbs,10,work,Fit like a glove!,hourglass,LOVE IT!!! First Item Im thinking of buying!,...,"5' 9""",8,42,"May 18, 2016",0.500000,0.400000,0.6476,positive,0.7964,positive
192458,fit,118398,32c,682043,100lbs,10,work,The pattern contrast on this dress is really s...,petite,LOVE it!,...,"5' 1""",4,29,"September 30, 2016",0.259583,0.598750,0.8951,positive,0.7371,positive
192459,fit,47002,36a,683251,135lbs,6,everyday,"Like the other DVF wraps, the fit on this is f...",straight & narrow,"Loud patterning, flattering fit",...,"5' 8""",8,31,"March 4, 2016",0.178409,0.563636,0.4072,positive,0.5859,positive
192460,fit,961120,36c,126335,165lbs,10,wedding,This dress was PERFECTION. it looked incredib...,pear,loved this dress it was comfortable and photog...,...,"5' 6""",16,31,"November 25, 2015",0.455556,0.726667,0.9623,positive,0.8716,positive


In [9]:
def is_column_utf8(df, column):
    try:
        df[column].astype(str).apply(lambda x: x.encode('utf-8').decode('utf-8'))
        return True
    except UnicodeEncodeError:
        return False

# Example usage with a DataFrame 'df'
for column in rent.columns:
    if is_column_utf8(rent, column):
        print(f"Column '{column}' is UTF-8 compatible.")
    else:
        print(f"Column '{column}' is NOT UTF-8 compatible.")

Column 'fit' is UTF-8 compatible.
Column 'user_id' is UTF-8 compatible.
Column 'bust size' is UTF-8 compatible.
Column 'item_id' is UTF-8 compatible.
Column 'weight' is UTF-8 compatible.
Column 'rating' is UTF-8 compatible.
Column 'rented for' is UTF-8 compatible.
Column 'review_text' is NOT UTF-8 compatible.
Column 'body type' is UTF-8 compatible.
Column 'review_summary' is NOT UTF-8 compatible.
Column 'category' is UTF-8 compatible.
Column 'height' is UTF-8 compatible.
Column 'size' is UTF-8 compatible.
Column 'age' is UTF-8 compatible.
Column 'review_date' is UTF-8 compatible.
Column 'TextBlob_polarity' is UTF-8 compatible.
Column 'TextBlob_subjectivity' is UTF-8 compatible.
Column 'nltk_sentiment_compound_score' is UTF-8 compatible.
Column 'nltk_sentiment_category' is UTF-8 compatible.
Column 'nltk_sentiment_compound_score_summary' is UTF-8 compatible.
Column 'nltk_sentiment_category_summary' is UTF-8 compatible.


In [10]:
def replace_non_utf8(text):
    return text.encode('utf-8', 'replace').decode('utf-8')

rent['review_text'] = rent['review_text'].apply(lambda x: replace_non_utf8(str(x)))
rent['review_summary'] = rent['review_summary'].apply(lambda x: replace_non_utf8(str(x)))


In [11]:
rent.to_csv('rent_sentiment.csv',index=False)

In [8]:
df = pd.read_csv('rent_sentiment.csv')
df

,fit,user_id,bust size,item_id,weight,rating,rented for,review_text,body type,review_summary,...,height,size,age,review_date,TextBlob_polarity,TextBlob_subjectivity,nltk_sentiment_compound_score,nltk_sentiment_category,nltk_sentiment_compound_score_summary,nltk_sentiment_category_summary
0,fit,420272,34d,2260466,137lbs,10,vacation,An adorable romper! Belt and zipper were a lit...,hourglass,So many compliments!,...,"5' 8""",14,28.0,"April 20, 2016",0.181548,0.623810,0.9453,positive,0.5057,positive
1,fit,273551,34b,153475,132lbs,10,other,I rented this dress for a photo shoot. The the...,straight & narrow,I felt so glamourous!!!,...,"5' 6""",12,36.0,"June 18, 2013",0.360667,0.686667,0.8991,positive,0.6333,positive
2,fit,360448,NaN,1063761,NaN,10,party,This hugged in all the right places! It was a ...,NaN,It was a great time to celebrate the (almost) ...,...,"5' 4""",4,116.0,"December 14, 2015",0.489762,0.570476,0.9333,positive,0.8316,positive
3,fit,909926,34c,126335,135lbs,8,formal affair,I rented this for my company's black tie award...,pear,Dress arrived on time and in perfect condition.,...,"5' 5""",8,34.0,"February 12, 2014",0.213056,0.461556,0.9559,positive,0.5719,positive
4,fit,151944,34b,616682,145lbs,10,wedding,I have always been petite in my upper body and...,athletic,Was in love with this dress !!!,...,"5' 9""",12,27.0,"September 26, 2016",-0.054028,0.320417,0.6948,positive,0.7249,positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192457,fit,66386,34dd,2252812,140lbs,10,work,Fit like a glove!,hourglass,LOVE IT!!! First Item Im thinking of buying!,...,"5' 9""",8,42.0,"May 18, 2016",0.500000,0.400000,0.6476,positive,0.7964,positive
192458,fit,118398,32c,682043,100lbs,10,work,The pattern contrast on this dress is really s...,petite,LOVE it!,...,"5' 1""",4,29.0,"September 30, 2016",0.259583,0.598750,0.8951,positive,0.7371,positive
192459,fit,47002,36a,683251,135lbs,6,everyday,"Like the other DVF wraps, the fit on this is f...",straight & narrow,"Loud patterning, flattering fit",...,"5' 8""",8,31.0,"March 4, 2016",0.178409,0.563636,0.4072,positive,0.5859,positive
192460,fit,961120,36c,126335,165lbs,10,wedding,This dress was PERFECTION. it looked incredib...,pear,loved this dress it was comfortable and photog...,...,"5' 6""",16,31.0,"November 25, 2015",0.455556,0.726667,0.9623,positive,0.8716,positive


In [9]:
df[['review_text', 'review_summary']].isna().sum()

review_text        68
review_summary    347
dtype: int64

In [10]:
def replace_with_int(value):
    if value == 'fit':
        return 1
    else:
        return 0

In [11]:
df['fit'] = df['fit'].apply(replace_with_int)

In [12]:
df['nltk_sentiment_category'].value_counts()

nltk_sentiment_category
positive    180885
negative      6579
neutral       4998
Name: count, dtype: int64

## Example comparing two sentiment analysis

In [13]:
df['review_text'].iloc[192459]

'Like the other DVF wraps, the fit on this is fantastic, albeit the collar drops a little low depending on how you cinch the wrap. This pattern looks great from far away, but close up it is very in-your-face. Not my favorite DVF wrap due to that, but may be a personal preference. '

In [14]:
df[['TextBlob_polarity','nltk_sentiment_compound_score']].iloc[192459]

TextBlob_polarity                0.178409
nltk_sentiment_compound_score    0.407200
Name: 192459, dtype: float64

In [15]:
df['review_summary'].iloc[192459]

'Loud patterning, flattering fit'

In [16]:
df[['nltk_sentiment_compound_score_summary']].iloc[192459]

nltk_sentiment_compound_score_summary    0.5859
Name: 192459, dtype: float64

In [17]:
df['review_text'].iloc[192460]  # due to the captialize vader better so we use vader

"This dress was PERFECTION.  it looked incredible in photos and we LOVED wearing it. every girl in the wedding party looked incredible and we are so happy that our beautiful Bride chose this dress. I wish i didn't need to return it!\nThe only issue was for 2 of the girls the stitch was missing at the bust which made for a VERY low cut V neck.  each girl in the wedding party ended up wearing their smaller of their 2 size options rented.  "

In [18]:
df[['TextBlob_polarity','nltk_sentiment_compound_score']].iloc[192460]

TextBlob_polarity                0.455556
nltk_sentiment_compound_score    0.962300
Name: 192460, dtype: float64

In [19]:
df['review_summary'].iloc[192460] 

'loved this dress it was comfortable and photographed amazingly well!'

In [20]:
df[['nltk_sentiment_compound_score_summary']].iloc[192460]

nltk_sentiment_compound_score_summary    0.8716
Name: 192460, dtype: float64

# prediction on rating (get mse)

In [21]:
df = df.dropna(subset=['review_text','review_summary'])  # drop na

In [33]:
df.head()

,fit,user_id,bust size,item_id,weight,rating,rented for,review_text,body type,review_summary,...,height,size,age,review_date,TextBlob_polarity,TextBlob_subjectivity,nltk_sentiment_compound_score,nltk_sentiment_category,nltk_sentiment_compound_score_summary,nltk_sentiment_category_summary
0,1,420272,34d,2260466,137lbs,10,vacation,An adorable romper! Belt and zipper were a lit...,hourglass,So many compliments!,...,"5' 8""",14,28.0,"April 20, 2016",0.181548,0.623810,0.9453,positive,0.5057,positive
1,1,273551,34b,153475,132lbs,10,other,I rented this dress for a photo shoot. The the...,straight & narrow,I felt so glamourous!!!,...,"5' 6""",12,36.0,"June 18, 2013",0.360667,0.686667,0.8991,positive,0.6333,positive
2,1,360448,NaN,1063761,NaN,10,party,This hugged in all the right places! It was a ...,NaN,It was a great time to celebrate the (almost) ...,...,"5' 4""",4,116.0,"December 14, 2015",0.489762,0.570476,0.9333,positive,0.8316,positive
3,1,909926,34c,126335,135lbs,8,formal affair,I rented this for my company's black tie award...,pear,Dress arrived on time and in perfect condition.,...,"5' 5""",8,34.0,"February 12, 2014",0.213056,0.461556,0.9559,positive,0.5719,positive
4,1,151944,34b,616682,145lbs,10,wedding,I have always been petite in my upper body and...,athletic,Was in love with this dress !!!,...,"5' 9""",12,27.0,"September 26, 2016",-0.054028,0.320417,0.6948,positive,0.7249,positive


In [16]:
def prediction_rating_mse(models, x):
    X = df[[x]]  # Features (independent variables)
    y = df['rating']  # Target variable

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = models
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Round predictions to the nearest integer
    
    # y_pred_rounded = np.rint(y_pred)

    # Calculate Mean Squared Error
    mse = mean_squared_error(y_test, y_pred) #y_pred_rounded
    return mse

## Textblob score on review_text VS rating

In [18]:
# Linear regression
mse = prediction_rating_mse(LinearRegression(),'TextBlob_polarity')
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 1.9074186871700884


In [19]:
# DecisionTree 
mse = prediction_rating_mse(DecisionTreeRegressor(),'TextBlob_polarity')
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 2.4625162699750422


## sentiment score on review_text VS rating

In [59]:
# Linear regression
mse = prediction_rating_mse(LinearRegression(),'nltk_sentiment_compound_score')
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 1.8607394543868905


In [55]:
# DecisionTree 
mse = prediction_rating_mse(DecisionTreeRegressor(),'nltk_sentiment_compound_score')
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 2.1612567352994767


## sentiment score on review summary VS rating

In [64]:
# Linear regression
mse = prediction_rating_mse(LinearRegression(),'nltk_sentiment_compound_score_summary')
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 1.882953206471891


In [65]:
# DecisionTree 
mse = prediction_rating_mse(DecisionTreeRegressor(),'nltk_sentiment_compound_score_summary')
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 1.8151811331281478


## bag of words VS rating

In [51]:
# BoW Vectorization
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['review_summary'])
y = df['rating']
# Splitting Data
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.2, random_state=42)

# Training a Classifier
clf = LinearRegression()
clf.fit(X_train, y_train)

# Predicting and Evaluating
y_pred = clf.predict(X_test)
mse = mean_squared_error(y_test, y_pred) 
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 1.5832830868461587


## tfidf on review text VS rating

In [26]:
tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(df['review_text'])
y = df['rating'] 

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Round predictions to the nearest integer
y_pred_rounded = np.rint(y_pred)

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred_rounded)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 1.7211130489106385


## tfidf on review summary VS rating

In [29]:
tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(df['review_summary'])
y = df['rating'] 

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)
y_pred = model.predict(X_test)
model = LinearRegression()
model.fit(X_train, y_train)
# Round predictions to the nearest integer
y_pred_rounded = np.rint(y_pred)

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred_rounded)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 1.5926542936720722


# prediction on fit

## sentiment score on review_text VS fit

In [30]:
X = df[['nltk_sentiment_compound_score']]  # Features (independent variables)
y = df['fit']  # Target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy}")

Model Accuracy: 0.7385532446573132


## sentiment score on review_summary VS fit

In [32]:
X = df[['nltk_sentiment_compound_score_summary']]  # Features (independent variables)
y = df['fit']  # Target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy}")

Model Accuracy: 0.7387354556576515


## tfidf score on review text VS fit

In [41]:
tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(df['review_text'])
y = df['fit'] 

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy}")

Model Accuracy: 0.8134159356534867


## tfidf score on review summary VS fit

In [38]:
tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(df['review_summary'])
y = df['fit'] 

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy}")

Model Accuracy: 0.7626831871306974
